In [3]:
import xgboost as xgb
import seaborn as sns 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

In [4]:
warnings.filterwarnings("ignore")

In [5]:
diamonds = sns.load_dataset("diamonds")

In [7]:
X = diamonds.drop("cut", axis=1)
y = diamonds[["cut"]]

In [8]:
y_encoded = OrdinalEncoder().fit_transform(y)

In [11]:
cats = X.select_dtypes(exclude=np.number).columns.tolist()

In [13]:
for col in cats:
    X[col] = X[col].astype("category")

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, random_state=61, stratify=y_encoded)

In [15]:
dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [17]:
params = {"objective": "multi:softprob", "num_class": 5}
n = 1000

resutls = xgb.cv(
    params,dtrain_clf,
    num_boost_round=n,
    nfold=5,
    metrics=["mlogloss","auc","merror"],
)

In [18]:
resutls.keys()

Index(['train-mlogloss-mean', 'train-mlogloss-std', 'train-auc-mean',
       'train-auc-std', 'train-merror-mean', 'train-merror-std',
       'test-mlogloss-mean', 'test-mlogloss-std', 'test-auc-mean',
       'test-auc-std', 'test-merror-mean', 'test-merror-std'],
      dtype='object')

In [19]:
resutls["test-auc-mean"].max()

0.9407864735158862

In [21]:
resutls["test-merror-mean"].min()

0.2018291929304165